In [3]:
import pickle
import pandas as pd
import numpy as np

import torchaudio

In [4]:
label_dir = "/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/gdance_labels"
test_labels = pd.read_csv(f'{label_dir}/test_labels.csv')
train_labels = pd.read_csv(f'{label_dir}/train_labels.csv')
val_labels = pd.read_csv(f'{label_dir}/val_labels.csv')

In [5]:
print(train_labels.shape, test_labels.shape, val_labels.shape)
train_labels.head()

(1309, 3) (154, 3) (161, 3)


,id,music_genre,dance_style
0,-4yoUMiBwXg_01_0_960,Pop,Commercial
1,-4yoUMiBwXg_04_0_296,Pop,Commercial
2,-4yoUMiBwXg_05_0_1110,Pop,Commercial
3,-4yoUMiBwXg_06_0_741,Pop,Commercial
4,-FXdDRM4lC0_01_22_900,Pop,Zumba


In [6]:
id = '_P-JWcq1ewI_01_187_1358'
# id = '_P-4yoUMiBwXg_05_0_1110'
id = '-4yoUMiBwXg_04_0_296'

motion = f'/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/gdance/motions_smpl/{id}.pkl'
song = f'/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/gdance/musics/{id}.wav'

motion = pickle.load(open(motion,"rb"))
print(motion.keys())
for k in motion.keys():
    if k != 'meta':
        print(k, motion[k].shape)

wav, sr = torchaudio.load(song)
wav_secs = wav.shape[1]/sr
print(wav.shape, sr, wav_secs)
print(motion['smpl_poses'].shape[1]/wav_secs )

dict_keys(['smpl_poses', 'root_trans', 'smpl_betas', 'meta'])
smpl_poses (3, 296, 72)
root_trans (3, 296, 3)
smpl_betas (3, 296, 10)
torch.Size([2, 435200]) 44100 9.868480725623582
29.99448529411765


In [34]:
import os
import trimesh
from smplx import SMPL
# import open3d as o3d
import torch
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FFMpegWriter
import matplotlib.animation as animation
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.audio.AudioClip import AudioArrayClip  # Corrected import
import moviepy as mpy

def trim_data(smpl_poses, root_trans, smpl_betas, audio, fps, max_length, mono=False):
    num_frames = smpl_poses.shape[1]
    sample_rate = audio.shape[-1] // (num_frames / fps)

    if num_frames >= max_length * fps:
        # Split into segments of max_length
        num_segments = num_frames // (max_length * fps)
        segments = []
        for i in range(num_segments):
            start = i * max_length * fps
            end = min((i + 1) * max_length * fps, num_frames)

            segment_poses = smpl_poses[:, start:end]
            segment_trans = root_trans[:, start:end]
            segment_betas = smpl_betas[:, start:end]

            start_sample = int(start * sample_rate // fps)
            end_sample = int(end * sample_rate // fps)
            segment_audio = audio[:, start_sample:end_sample]

            if mono:
                segment_audio = segment_audio.mean(dim=0, keepdim=True)

            segments.append((segment_poses, segment_trans, segment_betas, segment_audio))
    # else:
    #     segment_poses = smpl_poses
    #     segment_trans = root_trans
    #     segment_betas = smpl_betas
    #     segment_audio = audio

    #     if mono:
    #         segment_audio = segment_audio.mean(dim=0, keepdim=True)

    #     segments = [(segment_poses, segment_trans, segment_betas, segment_audio)]

    return segments

def visualize_and_save_smpl2D(pose, trans, betas, segment_audio, smpl_model_path, output_video_path, fps=30):
    num_subjects, num_frames = pose.shape[0], pose.shape[1]
    smpl = SMPL(model_path=smpl_model_path, batch_size=num_subjects)
    betas = betas[:, 0, :]

    joint_connections = [(19, 21), (19, 17), (16, 18), (18, 20), (1,4), (2,5), (5,8), (4,7),
                         (0,2), (0,1), (0,3), (17,14), (16,13), (3,6),(6,9), (14,9), (13,9)]
    # create a set of all value sin joint conidtions
    joint_connections = set(joint_connections)

    fig, ax = plt.subplots()
    joint_sequence = []

    def update(frame):
        ax.clear()
        ax.set_xlim([-2, 2])
        ax.set_ylim([-2, 2])

        for n in range(num_subjects):
            global_orient = torch.tensor(pose[n, frame, :3].reshape(1, 3), dtype=torch.float32)
            body_pose = torch.tensor(pose[n, frame, 3:].reshape(1, -1), dtype=torch.float32)
            transl = torch.tensor(trans[n, frame].reshape(1, 3), dtype=torch.float32)
            betas_tensor = torch.tensor(betas[n].reshape(1, -1), dtype=torch.float32)

            joints = smpl(global_orient=global_orient, body_pose=body_pose, transl=transl, betas=betas_tensor).joints.detach().numpy().squeeze()
            joint_sequence.append(joints)

            # Project to 2D by using only X and Y coordinates
            ax.scatter(joints[:, 0], joints[:, 1], marker='o', s=5)

            # Annotate each joint with its index
            # for i, (x, y) in enumerate(joints[:, :2]):
                
            #     if i not in joint_connections and i // 2 != 0 or i == 0:
            #         ax.text(x, y, str(i), color="red", fontsize=8)

            # Draw lines between connected joints
            for start, end in joint_connections:
                ax.plot([joints[start, 0], joints[end, 0]],
                        [joints[start, 1], joints[end, 1]], 'b')

    ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=1000/fps)

    # video_file_path = os.path.join(output_video_path, "debug_render.mp4")
    # print(f"Video saved to {video_file_path}")
    # ani.save(video_file_path, writer='ffmpeg', fps=fps)
    
    # # Convert audio tensor to numpy array
    # segment_audio_np = segment_audio.numpy().astype(np.float32)
    
    # # Reshape the audio for single channel (mono) if required
    # if segment_audio_np.ndim == 2:
    #     segment_audio_np = segment_audio_np.T  # Transpose to get the correct shape

    # # Add audio to the video using moviepy
    # video_clip = VideoFileClip(video_file_path)
    # audio_clip = AudioArrayClip(segment_audio_np, fps=44100)  # You might need to adjust the fps
    # video_clip_with_audio = video_clip.set_audio(audio_clip)
    # # video_clip_with_audio.write_videofile(video_file_path, codec='libx264', audio_codec='aac')

    # plt.close()

    # # save the joints as a numpy file for later use
    # joints_file_path = video_file_path.replace("debug_render.mp4", "joints.npy")
    # joint_sequence = np.array(joint_sequence)
    # # np.save(joints_file_path, joint_sequence)
    return ani

In [40]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


smpl_poses = motion['smpl_poses']
root_trans = motion['root_trans']
smpl_betas = motion['smpl_betas']
audio = wav

max_length = 3
mono = True
fps = 30
segments = trim_data(smpl_poses, root_trans, smpl_betas, audio, fps, max_length, mono)
output_dir = '/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/utils/test'
smpl_path = "/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/building_dataset_tools/AIOZ_Processing/models/smpl"

animations = []
for i, (segment_poses, segment_trans, segment_betas, segment_audio) in enumerate(segments):
    output_segment_dir = os.path.join(output_dir, f"{i}")
    os.makedirs(output_segment_dir, exist_ok=True)
    animations.append(visualize_and_save_smpl2D(segment_poses, segment_trans, segment_betas, segment_audio, smpl_path, output_segment_dir))

# visualize an animation
ani = animations[0]
display(HTML(ani.to_html5_video()))

in loop


<IPython.core.display.Javascript object>

in loop


<IPython.core.display.Javascript object>

in loop


<IPython.core.display.Javascript object>

## Test Dataset Construction 

In [65]:
p = "/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/samples2/-4yoUMiBwXg_02_0_1050_2/joints.npy"
np.load(p).shape

(273, 1, 45, 3)

In [92]:
from torch.utils.data import DataLoader, Dataset
import pickle as pkl
import librosa

class DanceToMusic_SMPL(Dataset):
    def __init__(self, directory, encoder = None, sample_rate=24000, device = torch.device("cpu"), num_samples = None, dnb=False):
        self.device = device
        self.raw_data = self._load_data(directory, sample_rate, num_samples, dnb)
        self.data = self._buildData(self.raw_data)
        if 'audio_codes' not in self.data.keys():
            self.encoder = encoder
            if encoder is not None:
                print("Encoding audio...")
                self.data = self._encodeAudio(encoder, self.data)

    def __len__(self):
        return len(self.data['poses'])

    def __getitem__(self, idx):
        # pose = self.data['poses'][idx]
        joints = self.data['joints'][idx]
        joint_mask = self.data['joint_padding_mask'][idx]
        wav = self.data['wavs'][idx]
        wav_path = self.data['wav_paths'][idx]
        sample_rate = self.data['sample_rate'][idx]
        # pose_mask = self.data['pose_padding_mask'][idx]
        wav_mask = self.data['audio_padding_mask'][idx]
        vid_path = self.data['video_paths'][idx]

        if 'audio_codes' not in self.data.keys():
            return [joints, joint_mask, wav, wav_mask, wav_path, vid_path, sample_rate]
        
        audio_codes = self.data['audio_codes'][idx]
        return [audio_codes, joints, joint_mask, wav, wav_mask, wav_path, vid_path, sample_rate]

    def _load_data(self, directory, sr, num_samples, dnb):
        poses = []
        joints = []
        wavs = []
        wav_paths = []
        sample_rate = []
        count_loaded_sample = 0
        audio_codes = []
        video_paths = []
        for root, dirs, files in os.walk(directory):
            for d in dirs:
                if 'error' not in d and 'spleeter' not in os.path.join(root,d):
                    joint_path = os.path.join(root, d, "joints.npy")
                    joint_arr = np.load(joint_path) 

                    pose_path = os.path.join(root, d, "poses.pkl")
                    motion = pkl.load(open(pose_path, "rb"))

                    if len(joint_path) > 0:
                        if dnb:
                            wav_path = os.path.join(root, d, f"audio_drum_and_bass.wav")
                        else:
                            wav_path = os.path.join(root, d, "audio.wav")
                        
                        vid_path = os.path.join(root, d, f"debug_render.mp4")
                        video_paths.append(vid_path)

                        audio_code_path = os.path.join(root, d, f"audio_code.npy")
                        if os.path.exists(audio_code_path):
                            audio_code = np.load(audio_code_path)
                            audio_codes.append(audio_code)

                        # poses.append(self._buildPoses(pose_dir_path))
                        joints.append(joint_arr)
                        poses.append(motion)
                        wav, sr = librosa.load(wav_path, sr=sr)
                        sample_rate.append(sr)
                        wav_paths.append(wav_path)
                        wavs.append(wav)
                        count_loaded_sample += 1
                        if count_loaded_sample == num_samples:
                            break

        ret = {
            "poses": poses,
            "joints": joints,
            "wavs": wavs,
            "wav_paths": wav_paths,
            "sample_rate": sample_rate,
            "video_paths":video_paths
        }
        if len(audio_codes) > 0:
            ret['audio_codes'] = audio_codes
        return ret

    # precomuting the encoding of the adio and adding a start token at the beginning of each audio code sequence
    def _encodeAudio(self, encoder, data):
        audio_codes = []
        for i, wav in enumerate(data['wavs']):
            wav = wav.unsqueeze(0).to(self.device)
            audio_padding_mask = data['audio_padding_mask'].to(self.device)
            encoding = encoder.encode(wav, audio_padding_mask[i].unsqueeze(0))
            # one_audio_code = encoding['audio_codes'].view(1,1,-1)
            one_audio_code = encoding['audio_codes'].view(1,-1,2)
            one_audio_code = one_audio_code.squeeze(0)
            # # Adding a start token at the beginning of each audio code sequence
            audio_code_with_start = torch.cat((torch.zeros(1,2).to(self.device), one_audio_code), dim=0)
            audio_codes.append(audio_code_with_start.to(self.device))
        data['audio_codes'] = audio_codes
        return data

    def _buildData(self, raw_data, movement_threshold=0.1, keypoints_threshold=4, frame_threshold = 0.2):
        # Handle poses
        # poses = [torch.tensor(p) for p in raw_data['poses']]
        # max_pos_seq_len = max(p.shape[0] for p in poses)

        joints = [torch.tensor(j) for j in raw_data['joints']]
        max_joint_seq_len = max(j.shape[1] for j in joints)
        
        # Handle audio
        wavs = [torch.tensor(w) for w in raw_data['wavs']]
        max_audio_len = max(w.shape[0] for w in wavs)
        
        num_samples = len(wavs)
        max_num_subjects = max(j.shape[0] for j in joints)
        
        # Padding for poses
        # padded_poses = torch.zeros(num_samples, max_pos_seq_len, poses[0].shape[1], poses[0].shape[2])
        # pose_padding_mask = torch.zeros(num_samples, max_pos_seq_len, dtype=torch.bool)
        
        # Fill the tensors for poses
        # for i, seq in enumerate(poses):
        #     n = seq.size(0)
        #     padded_poses[i, :n, :, :] = seq
        #     pose_padding_mask[i, :n] = 1  # Set mask to 1 for non-padded values

        # Padding for joints
        padded_joints = torch.zeros(num_samples, max_num_subjects, max_joint_seq_len, joints[0].size(2), joints[0].size(3))
        joint_padding_mask = torch.zeros(num_samples, max_num_subjects, max_joint_seq_len, dtype=torch.bool)

        # Fill the tensors for joints
        for i, seq in enumerate(joints):
            num_subjects = seq.size(0)
            n = seq.size(1)
            padded_joints[i, :num_subjects, :n, :, :] = seq
            joint_padding_mask[i, :num_subjects, :n] = 1

        # Padding for audio
        padded_audio = torch.zeros(num_samples, max_audio_len)
        audio_padding_mask = torch.zeros(num_samples, max_audio_len, dtype=torch.bool)

        # Fill the tensors for audio
        for i, audio in enumerate(wavs):
            n = audio.size(0)
            padded_audio[i, :n] = audio
            audio_padding_mask[i, :n] = 1  # Set mask to 1 for non-padded values

        data = {
            'wav_paths': raw_data['wav_paths'],
            'sample_rate': raw_data['sample_rate'],  # Assuming this is the same for all samples
            'video_paths': raw_data['video_paths'],
            # 'poses': padded_poses.to(self.device),
            # 'pose_padding_mask': pose_padding_mask.to(self.device),
            'joints': padded_joints.to(self.device),
            'joint_padding_mask': joint_padding_mask.to(self.device),
            'wavs': padded_audio.unsqueeze(1).to(self.device),
            'audio_padding_mask': audio_padding_mask.to(self.device)
        }
        if 'audio_codes' in raw_data.keys():
            data['audio_codes'] = raw_data['audio_codes']
        
        return data

    def _buildPoses(self, directory):
        poses = []
        for root, dirs, files in os.walk(directory):
            for f in files:
                if '.npy' in f:
                    pose = np.load(os.path.join(root, f))
                    poses.append(pose)
        return np.array(poses)

    
from transformers import EncodecModel

# assign GPU or CPU
if torch.backends.mps.is_available():
    device = torch.device("mps")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model_id = "facebook/encodec_24khz"
encodec_model = EncodecModel.from_pretrained(model_id)
encodec_model.to(device)
codebook_size = encodec_model.quantizer.codebook_size
sample_rate = 24000

data_dir = "/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/test_smaple"
dataset = DanceToMusic_SMPL(data_dir, encoder = encodec_model, sample_rate = sample_rate, device=device)
audio_codes, joints, joint_mask, wav, wav_mask, wav_path, vid_path, sample_rate = dataset[0]
audio_codes.shape, joints.shape, joint_mask.shape, wav.shape, wav_mask.shape, wav_path, vid_path, sample_rate

Encoding audio...


(torch.Size([227, 2]),
 torch.Size([3, 91, 45, 3]),
 torch.Size([3, 91]),
 torch.Size([1, 72032]),
 torch.Size([72032]),
 '/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/test_smaple/-4yoUMiBwXg_02_0_1050_10/audio.wav',
 '/Users/azeez/Documents/pose_estimation/DanceToMusicApp/ml/data/aioz/test_smaple/-4yoUMiBwXg_02_0_1050_10/debug_render.mp4',
 24000)